## Imports and dataset loading

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torchvision import models
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import gc
import os

# clear cuda memory and collect garbage -> free up memory
gc.collect()
torch.cuda.empty_cache()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # check if gpu is available
print('Device: ',device) 

x=np.load('datasets/scaled_spec_resampled_array.npy')
y=np.load('datasets/labels_array.npy')-1 # labels start from 1, we want them to start from 0
x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2]) # add channel dimension for CNN

print(x.shape, y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) 

class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Device:  cuda:0
(1754, 1, 2048, 80) (1754,)


custom training and testing funcs

In [2]:
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = running_loss / len(test_loader)
    val_acc = correct / total
    return val_loss, val_acc

def train(model, train_loader, test_loader, criterion, optimizer, epochs, checkpointname):
    model.train()
    epoch_bar = tqdm(range(epochs), position=0)

    best_val_acc = 0
    val_loss, val_acc = test(model, test_loader, criterion)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), checkpointname)

    for epoch in epoch_bar:
        running_loss = 0.0
        batch_bar=tqdm(enumerate(train_loader, 0), total=len(train_loader), position=1, leave=False)
        for i, data in batch_bar:

        #for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            batch_bar.set_description('Train loss: %.3f' % (loss.item()))

        train_loss = running_loss / len(train_loader)
        epoch_bar.set_description('Train loss: %.3f' % train_loss)
        val_loss, val_acc = test(model, test_loader, criterion)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), checkpointname)

        print('Epoch: %d, Train Loss: %.3f, Val Loss: %.3f, Val Acc: %.3f' % (epoch, train_loss, val_loss, val_acc))
    return model


## Train ResNet18 and ResNet34

In [21]:
resnet18 = models.resnet18(pretrained=True)
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) #change input channels to 1 to use single layer spectrogram images
resnet18.fc = nn.Linear(in_features=512, out_features=6, bias=True) #change output classes to 6   

#check for checkpoint save and load resnet18.pth
if os.path.exists('checkpoints/resnet18.pth'):
    resnet18.load_state_dict(torch.load('checkpoints/resnet18.pth'))
    print('Model loaded')
else:
    print('No model found, loading pretrained model')

resnet18 = resnet18.to(device)

#freeze every layer except the first conv1 layer and the last fc layer, to adapt the model to our data
for name, param in resnet18.named_parameters():
    if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
        param.requires_grad = False
    else:
        param.requires_grad = True

optimizer = optim.Adam(resnet18.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
resnet18=train(resnet18, train_loader, test_loader, criterion, optimizer, 100, 'checkpoints/resnet18.pth')

Model loaded


Train loss: 0.657:   1%|          | 1/100 [00:09<15:48,  9.58s/it]

Epoch: 0, Train Loss: 0.657, Val Loss: 0.662, Val Acc: 0.766


Train loss: 1.183:   2%|▏         | 2/100 [00:17<14:10,  8.68s/it]

Epoch: 1, Train Loss: 1.183, Val Loss: 0.581, Val Acc: 0.786


...

Epoch: 97, Train Loss: 42.862, Val Loss: 0.473, Val Acc: 0.832


Train loss: 43.222:  99%|█████████▉| 99/100 [15:06<00:09,  9.36s/it]

Epoch: 98, Train Loss: 43.222, Val Loss: 0.542, Val Acc: 0.801


Train loss: 43.615: 100%|██████████| 100/100 [15:16<00:00,  9.16s/it]

Epoch: 99, Train Loss: 43.615, Val Loss: 0.534, Val Acc: 0.801


In [31]:
torch.cuda.empty_cache()

In [32]:
resnet34 = models.resnet34(pretrained=True)
resnet34.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet34.fc = nn.Linear(in_features=512, out_features=6, bias=True)    

#check for checkpoint save and load resnet34.pth
if os.path.exists('checkpoints/resnet34.pth'):
    resnet34.load_state_dict(torch.load('checkpoints/resnet34.pth'))
    print('Model loaded')
else:
    print('No model found, loading pretrained model')

resnet34 = resnet34.to(device)

#freeze every layer except the first conv1 and last fc layers of resnet50 to adapt the model to our data
for name, param in resnet34.named_parameters():
    if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
        param.requires_grad = False
    else:
        param.requires_grad = True

optimizer = optim.Adam(resnet34.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

resnet34 = train(resnet34, train_loader, test_loader, criterion, optimizer, 100, 'checkpoints/resnet34.pth')

Model loaded


Train loss: 0.917:   1%|          | 1/100 [00:15<26:03, 15.80s/it]

Epoch: 0, Train Loss: 0.917, Val Loss: 0.859, Val Acc: 0.675


Train loss: 0.848:   2%|▏         | 2/100 [00:29<24:05, 14.75s/it]

Epoch: 1, Train Loss: 0.848, Val Loss: 0.854, Val Acc: 0.678


...

Epoch: 97, Train Loss: 0.798, Val Loss: 0.804, Val Acc: 0.672


Train loss: 0.780:  99%|█████████▉| 99/100 [26:07<00:16, 16.28s/it]

Epoch: 98, Train Loss: 0.780, Val Loss: 0.789, Val Acc: 0.715


Train loss: 0.766: 100%|██████████| 100/100 [26:23<00:00, 15.83s/it]

Epoch: 99, Train Loss: 0.766, Val Loss: 0.861, Val Acc: 0.658


## Training repeated with only the center portions of the spectrograms

In [33]:
x=np.load('datasets/scaled_spec_resampled_array.npy') #load the dataset
x=x[:,724:1324,:] #take only the 600 frequency bins around the centre of the spectrogram
y=np.load('datasets/labels_array.npy')-1 # labels start from 1, we want them to start from 0
x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2]) # add channel dimension for CNN

print(x.shape, y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

(1754, 1, 600, 80) (1754,)


In [34]:
resnet18 = models.resnet18(pretrained=True)
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet18.fc = nn.Linear(in_features=512, out_features=6, bias=True)    

resnet18 = resnet18.to(device)

#check for checkpoint save and load resnet18_smalldata.pth
if os.path.exists('checkpoints/resnet18_smalldata.pth'):
    resnet18.load_state_dict(torch.load('checkpoints/resnet18_smalldata.pth'))
    print('Model loaded')
else:
    print('No model found, loading pretrained model')

#freeze every layer except the first conv1 layer and the last fc layer, to adapt the model to our data
for name, param in resnet18.named_parameters():
    if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
        param.requires_grad = False
    else:
        param.requires_grad = True

optimizer = optim.Adam(resnet18.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

resnet18=train(resnet18, train_loader,test_loader, criterion, optimizer, 100, 'checkpoints/resnet18_smalldata.pth')

Model loaded


Train loss: 0.346:   1%|          | 1/100 [00:03<05:58,  3.62s/it]

Epoch: 0, Train Loss: 0.346, Val Loss: 0.384, Val Acc: 0.838


Train loss: 0.295:   2%|▏         | 2/100 [00:06<05:24,  3.31s/it]

Epoch: 1, Train Loss: 0.295, Val Loss: 0.381, Val Acc: 0.869


...

Epoch: 97, Train Loss: 0.225, Val Loss: 0.390, Val Acc: 0.858


Train loss: 0.190:  99%|█████████▉| 99/100 [05:07<00:03,  3.05s/it]

Epoch: 98, Train Loss: 0.190, Val Loss: 0.416, Val Acc: 0.835


Train loss: 0.196: 100%|██████████| 100/100 [05:10<00:00,  3.10s/it]

Epoch: 99, Train Loss: 0.196, Val Loss: 0.379, Val Acc: 0.843


In [35]:
torch.cuda.empty_cache()

In [36]:
resnet34 = models.resnet34(pretrained=True)
resnet34.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet34.fc = nn.Linear(in_features=512, out_features=6, bias=True)    

#check for checkpoint save and load resnet34.pth
if os.path.exists('checkpoints/resnet34_smalladata.pth'):
    resnet34.load_state_dict(torch.load('checkpoints/resnet34_smalladata.pth'))
    print('Model loaded')
else:
    print('No model found, loading pretrained model')

resnet34 = resnet34.to(device)

#freeze every layer except the first conv1 and last fc layers of resnet50
for name, param in resnet34.named_parameters():
    if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
        param.requires_grad = False
    else:
        param.requires_grad = True

optimizer = optim.Adam(resnet34.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

resnet34=train(resnet34, train_loader,test_loader, criterion, optimizer, 100, 'checkpoints/resnet34_smalladata.pth')

Model loaded


Train loss: 0.630:   1%|          | 1/100 [00:05<09:23,  5.69s/it]

Epoch: 0, Train Loss: 0.630, Val Loss: 0.645, Val Acc: 0.769


Train loss: 0.543:   2%|▏         | 2/100 [00:10<08:39,  5.30s/it]

Epoch: 1, Train Loss: 0.543, Val Loss: 0.643, Val Acc: 0.761


...

Epoch: 97, Train Loss: 0.449, Val Loss: 0.614, Val Acc: 0.772


Train loss: 0.444:  99%|█████████▉| 99/100 [08:20<00:05,  5.06s/it]

Epoch: 98, Train Loss: 0.444, Val Loss: 0.548, Val Acc: 0.809


Train loss: 0.463: 100%|██████████| 100/100 [08:25<00:00,  5.06s/it]

Epoch: 99, Train Loss: 0.463, Val Loss: 0.615, Val Acc: 0.769


## Training with subject fold

In [4]:
from sklearn.model_selection import GroupKFold

x = np.load('datasets/scaled_spec_resampled_array.npy')
y = np.load('datasets/labels_array.npy')-1
x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2]) # add channel dimension for CNN
subjects = np.load('datasets/subjects_array.npy')
activities = ['Walking', 'Sitting Down', 'Standing Up', 'Picking up an Object', 'Drinking Water', 'Falling']


print(f"Spectrograms (X) shape: {x.shape}")
print(f"Labels (Y) shape: {y.shape}")
print(f"Subjects shape: {subjects.shape}")

num_samples = x.shape[0]
num_classes = len(np.unique(y))
print(f"Number of samples: {num_samples}")
print(f"Number of classes: {num_classes}")

gkf = GroupKFold(n_splits=5)
splits = list(gkf.split(x, y, groups=subjects))

# Verification:
for i, (train_index, test_index) in enumerate(splits):
    print(f"\nFold {i+1}:")
    print(f"Length of training set: {len(train_index)} samples")
    print(f"Length of testing set (fold): {len(test_index)} samples")
    print(f"Unique subjects in fold {i+1}: {np.unique(subjects[test_index])}")

Spectrograms (X) shape: (1754, 1, 2048, 80)
Labels (Y) shape: (1754,)
Subjects shape: (1754,)
Number of samples: 1754
Number of classes: 6

Fold 1:
Length of training set: 1398 samples
Length of testing set (fold): 356 samples
Unique subjects in fold 1: [ 4 16 20 23 25 28 32 38 42 46 54 57 58 60 67]

Fold 2:
Length of training set: 1401 samples
Length of testing set (fold): 353 samples
Unique subjects in fold 2: [ 5  8  9 19 21 31 35 39 45 49 52 61 64 68 72]

Fold 3:
Length of training set: 1405 samples
Length of testing set (fold): 349 samples
Unique subjects in fold 3: [ 2  3  7 11 14 18 26 29 37 44 55 65 70 71]

Fold 4:
Length of training set: 1406 samples
Length of testing set (fold): 348 samples
Unique subjects in fold 4: [ 6 12 17 22 34 36 43 47 48 53 56 62 63 69]

Fold 5:
Length of training set: 1406 samples
Length of testing set (fold): 348 samples
Unique subjects in fold 5: [ 1 10 13 15 24 27 30 33 40 41 50 51 59 66]


In [5]:
for i, (train_index, test_index) in enumerate(splits):
    print(f"\nFold {i+1}:")
    resnet18_folds = models.resnet18(pretrained=True)
    resnet18_folds.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet18_folds.fc = nn.Linear(in_features=512, out_features=6, bias=True)
    resnet18_folds = resnet18_folds.to(device)

    if os.path.exists(f'checkpoints/resnet18_fold{i+1}.pth'):
        resnet18_folds.load_state_dict(torch.load(f'checkpoints/resnet18_fold{i+1}.pth'))
        print('Model loaded')
    else:
        print('No model found, loading pretrained model')

    for name, param in resnet18_folds.named_parameters():
        if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
            param.requires_grad = False
        else:
            param.requires_grad = True
    
    optimizer = optim.Adam(resnet18_folds.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    train_dataset = MyDataset(x_train, y_train)
    test_dataset = MyDataset(x_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    resnet18_folds = train(resnet18_folds, train_loader, test_loader, criterion, optimizer, 150, f'checkpoints/resnet18_fold{i+1}.pth')


Fold 1:


c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded


Train loss: 0.848:   1%|          | 1/150 [00:11<29:15, 11.78s/it]

Epoch: 0, Train Loss: 0.848, Val Loss: 0.820, Val Acc: 0.680


...

Epoch: 147, Train Loss: 0.407, Val Loss: 0.488, Val Acc: 0.845


Train loss: 0.389:  99%|█████████▉| 149/150 [22:38<00:09,  9.13s/it]

Epoch: 148, Train Loss: 0.389, Val Loss: 0.537, Val Acc: 0.819


Train loss: 0.400: 100%|██████████| 150/150 [22:47<00:00,  9.12s/it]

Epoch: 149, Train Loss: 0.400, Val Loss: 0.537, Val Acc: 0.833


In [6]:
for i, (train_index, test_index) in enumerate(splits):
    print(f"\nFold {i+1}:")
    resnet34_folds = models.resnet34(pretrained=True)
    resnet34_folds.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet34_folds.fc = nn.Linear(in_features=512, out_features=6, bias=True)
    resnet34_folds = resnet34_folds.to(device)

    if os.path.exists(f'checkpoints/resnet34_fold{i+1}.pth'):
        resnet34_folds.load_state_dict(torch.load(f'checkpoints/resnet34_fold{i+1}.pth'))
        print('Model loaded')
    else:
        print('No model found, loading pretrained model')

    for name, param in resnet34_folds.named_parameters():
        if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
            param.requires_grad = False
        else:
            param.requires_grad = True
    
    optimizer = optim.Adam(resnet34_folds.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    train_dataset = MyDataset(x_train, y_train)
    test_dataset = MyDataset(x_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    resnet34_folds = train(resnet34_folds, train_loader, test_loader, criterion, optimizer, 150, f'checkpoints/resnet34_fold{i+1}.pth')


Fold 1:


c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded


Train loss: 1.824:   1%|          | 1/150 [00:16<40:50, 16.45s/it]

Epoch: 0, Train Loss: 1.824, Val Loss: 1.463, Val Acc: 0.433


...

Epoch: 147, Train Loss: 0.828, Val Loss: 0.854, Val Acc: 0.704


Train loss: 0.859:  99%|█████████▉| 149/150 [37:12<00:15, 15.01s/it]

Epoch: 148, Train Loss: 0.859, Val Loss: 0.865, Val Acc: 0.710


Train loss: 0.857: 100%|██████████| 150/150 [37:26<00:00, 14.98s/it]

Epoch: 149, Train Loss: 0.857, Val Loss: 0.858, Val Acc: 0.721


## Subject folds, but with small data

In [7]:
from sklearn.model_selection import GroupKFold

x = np.load('datasets/scaled_spec_resampled_array.npy')
x=x[:,724:1324,:]
y = np.load('datasets/labels_array.npy')-1
x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2]) # add channel dimension for CNN
subjects = np.load('datasets/subjects_array.npy')
activities = ['Walking', 'Sitting Down', 'Standing Up', 'Picking up an Object', 'Drinking Water', 'Falling']


print(f"Spectrograms (X) shape: {x.shape}")
print(f"Labels (Y) shape: {y.shape}")
print(f"Subjects shape: {subjects.shape}")

num_samples = x.shape[0]
num_classes = len(np.unique(y))
print(f"Number of samples: {num_samples}")
print(f"Number of classes: {num_classes}")

gkf = GroupKFold(n_splits=5)
splits = list(gkf.split(x, y, groups=subjects))

# Verification:
for i, (train_index, test_index) in enumerate(splits):
    print(f"\nFold {i+1}:")
    print(f"Length of training set: {len(train_index)} samples")
    print(f"Length of testing set (fold): {len(test_index)} samples")
    print(f"Unique subjects in fold {i+1}: {np.unique(subjects[test_index])}")

Spectrograms (X) shape: (1754, 1, 600, 80)
Labels (Y) shape: (1754,)
Subjects shape: (1754,)
Number of samples: 1754
Number of classes: 6

Fold 1:
Length of training set: 1398 samples
Length of testing set (fold): 356 samples
Unique subjects in fold 1: [ 4 16 20 23 25 28 32 38 42 46 54 57 58 60 67]

Fold 2:
Length of training set: 1401 samples
Length of testing set (fold): 353 samples
Unique subjects in fold 2: [ 5  8  9 19 21 31 35 39 45 49 52 61 64 68 72]

Fold 3:
Length of training set: 1405 samples
Length of testing set (fold): 349 samples
Unique subjects in fold 3: [ 2  3  7 11 14 18 26 29 37 44 55 65 70 71]

Fold 4:
Length of training set: 1406 samples
Length of testing set (fold): 348 samples
Unique subjects in fold 4: [ 6 12 17 22 34 36 43 47 48 53 56 62 63 69]

Fold 5:
Length of training set: 1406 samples
Length of testing set (fold): 348 samples
Unique subjects in fold 5: [ 1 10 13 15 24 27 30 33 40 41 50 51 59 66]


In [8]:
for i, (train_index, test_index) in enumerate(splits):
    print(f"\nFold {i+1}:")
    resnet18_folds = models.resnet18(pretrained=True)
    resnet18_folds.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet18_folds.fc = nn.Linear(in_features=512, out_features=6, bias=True)
    resnet18_folds = resnet18_folds.to(device)

    if os.path.exists(f'checkpoints/resnet18_fold{i+1}_small.pth'):
        resnet18_folds.load_state_dict(torch.load(f'checkpoints/resnet18_fold{i+1}_small.pth'))
        print('Model loaded')
    else:
        print('No model found, loading pretrained model')

    for name, param in resnet18_folds.named_parameters():
        if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
            param.requires_grad = False
        else:
            param.requires_grad = True
    
    optimizer = optim.Adam(resnet18_folds.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    train_dataset = MyDataset(x_train, y_train)
    test_dataset = MyDataset(x_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    resnet18_folds = train(resnet18_folds, train_loader, test_loader, criterion, optimizer, 150, f'checkpoints/resnet18_fold{i+1}_small.pth')


Fold 1:


c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


No model found, loading pretrained model


Train loss: 2.319:   1%|          | 1/150 [00:03<08:55,  3.60s/it]

Epoch: 0, Train Loss: 2.319, Val Loss: 1.743, Val Acc: 0.180


...

Epoch: 147, Train Loss: 0.296, Val Loss: 0.481, Val Acc: 0.859


Train loss: 0.300:  99%|█████████▉| 149/150 [07:06<00:02,  2.86s/it]

Epoch: 148, Train Loss: 0.300, Val Loss: 0.514, Val Acc: 0.848


Train loss: 0.301: 100%|██████████| 150/150 [07:09<00:00,  2.87s/it]

Epoch: 149, Train Loss: 0.301, Val Loss: 0.560, Val Acc: 0.851


In [9]:
for i, (train_index, test_index) in enumerate(splits):
    print(f"\nFold {i+1}:")
    resnet34_folds = models.resnet34(pretrained=True)
    resnet34_folds.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet34_folds.fc = nn.Linear(in_features=512, out_features=6, bias=True)
    resnet34_folds = resnet34_folds.to(device)

    if os.path.exists(f'checkpoints/resnet34_fold{i+1}_small.pth')):
        resnet34_folds.load_state_dict(torch.load(f'checkpoints/resnet34_fold{i+1}_small.pth'))
    else:
        print('No model found, loading pretrained model')

    for name, param in resnet34_folds.named_parameters():
        if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
            param.requires_grad = False
        else:
            param.requires_grad = True
    
    optimizer = optim.Adam(resnet34_folds.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    train_dataset = MyDataset(x_train, y_train)
    test_dataset = MyDataset(x_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    resnet34_folds = train(resnet34_folds, train_loader, test_loader, criterion, optimizer, 150, f'checkpoints/resnet34_fold{i+1}_small.pth')

SyntaxError: unmatched ')' (1171301323.py, line 8)